In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import pickle


import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3,4,5,6,7,8 ])

os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf
from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf
from helpers import eval_function_new


In [2]:
PF_SIZE = 500


base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = PF_SIZE  # DONT CHANGE HERE!!
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
rets_full = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")


fixed_shrk_name = 'cov1Para'
opt_shrk_name = 'cov1Para'
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    fixed_shrk_data = pickle.load(f)
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\{opt_shrk_name}_factor-1.0_p{pf_size}.pickle", 'rb') as f:
    optimal_shrk_data = pickle.load(f)

with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_rawres_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    rawres_fixed_shrk_data = pickle.load(f)

# IMPORT FACTORS DATA AND PREPARE FOR FURTHER USE
factor_path = fr"{base_folder_path}\helpers"
factors = pd.read_csv(factor_path + "/all_factors.csv")
factors = factors.pivot(index="date", columns="name", values="ret")

# as our shrk data starts from 1980-01-15 our factors data should too
start_date = str(optimal_shrk_data['date'].iloc[0])
start_date = start_date[0:4] + '-' + start_date[4:6] + "-" + start_date[6:]
start_idx = np.where(factors.index == start_date)[0][0]
factors = factors.iloc[start_idx:start_idx+fixed_shrk_data.shape[0], :]

cov1para_shrk = optimal_shrk_data['shrk_factor'].values[5040:]

len_train = 5040
end_date = fixed_shrk_data.shape[0]
val_indices_correct = (len_train, end_date)
val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]


all_res = fixed_shrk_data.iloc[:, 2:].copy() * 100
all_factors = all_res.columns.astype(float).values
Y = all_res.idxmin(axis=1).values.astype(float)
opt_values = all_res.idxmin(axis=1).values.astype(float)[:-21]
opt_values = np.insert(arr=opt_values, obj=0, values=np.repeat(0.15, 21))
Y = np.array(re_hf.map_factors_to_preds(Y.reshape(-1), all_factors))
opt_values = np.array(re_hf.map_factors_to_preds(opt_values, all_factors))
opt_v3 = np.diag(all_res.loc[:, all_res.idxmin(axis=1).values])[:-21]
opt_v3 = np.insert(arr=opt_v3, obj=0, values=np.repeat(7.0, 21))

rolling_opt = pd.Series(opt_values).rolling(window=252, min_periods=1).mean().values
rolling_opt2 = pd.Series(opt_v3).rolling(window=252, min_periods=1).mean().values


ROLL_WINDOW = 1
ROLL_WINDOW_INPUTS = 252
rolling_opt = pd.Series(opt_values).rolling(window=ROLL_WINDOW_INPUTS, min_periods=1).mean().values
rolling_opt2 = pd.Series(opt_v3).rolling(window=ROLL_WINDOW_INPUTS, min_periods=1).mean().values

Y = pd.Series(Y).rolling(window=1, min_periods=1).mean().astype(np.int64).values
opt_values = pd.Series(opt_values).rolling(window=1, min_periods=1).mean().astype(np.int64).values

params = {
'pf_size' : pf_size,
'opt_values_factors' : opt_values,
'include_ts_momentum_var_allstocks': False,
'include_ts_momentum_allstocks': True,
'include_sample_covmat_trace': True,
'include_mean_of_correls': True,
'include_iqr': False,
'include_factors': False,
'include_ewma_year': False,
'include_ewma_month': True,
'include_ew_year_vola': False,
'include_ew_month_vola': True,
'include_allstocks_year_avgvola': True,
'include_allstocks_month_avgvola': False,
    'additional_inputs' : [opt_v3, rolling_opt, rolling_opt2, optimal_shrk_data['shrk_factor'].values.astype(np.float64)*100]  
}

X = re_hf.load_additional_train_data(**params)
X=X.astype(np.float64)


### define all needed data structures to use the imitation package

In [12]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete
from typing import Optional, Dict, Any

In [7]:
a = np.random.choice(range(5), 5)
print(a)
new = a[0]
a=a[1:]
print(new)
print(a)

[1][1:]

[2 0 2 3 1]
2
[0 2 3 1]


[]

In [18]:
a[2:].any()

True

In [10]:
if not [1]:
    print("empty")

In [21]:
class TimeSeriesEnv(gym.Env):
    """
    Exog. time-series; actions only affect rewards, not future obs.
    """
    def __init__(self, data: np.array, expert_actions: np.array, reward_fn: callable, train_size: np.array):
        self.state = None
        # save whole train data in env.
        self.data = data
        self.T, self.obs_dim = data.shape
        self.observation_space = Box(-np.inf, np.inf, shape=(self.obs_dim, ) )
        self.action_space = Discrete(n=101) # shrinkage between 0.0 and 1.0, 101 steps
        self.reward_fn = reward_fn
        self.expert_actions = expert_actions
        self.train_size = train_size
    
    def reset(self, seed=412, **kwargs):
        """
        reset env to an initial state, in our case start of t.s. or just random?
        """
        super().reset(seed=seed)
        self.training_order = np.random.choice(range(self.train_size), self.train_size)
        self.idx = self.training_order[0]
        #self.state = self.observation_space.sample()
        self.state = self.data[self.idx]
        return self.state, {}  # 'self.state' can be OOS standard deviation?
    
    def step(self, action):
        """
        from Gymnasium docs;
        Updates an environment with actions returning the next agent observation, 
        the reward for taking that actions, if the environment has terminated or 
        truncated due to the latest action and information from the environment 
        about the step, i.e. metrics, debug info.
        """
        cur_state = self.data[self.idx]
        reward = self.reward_fn([self.expert_actions[self.idx]], [action])
        # next state
        self.training_order = self.training_order[1:]
        if not self.training_order.any():
            done = True
            next_state = np.zeros_like(cur_state)
        else:
            done = False
            self.idx = self.training_order[1:]
            next_state = self.data[self.idx].copy()

        # terminate after one step hence --> can just return None for next_state?
        return next_state, reward, done, False, {}
    

from sklearn.metrics import mean_squared_error
# register, vectorize, and wrap env to work with imitation apackage
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.util.util import make_vec_env
gym.register(
    id="TimeSeriesEnv",
    entry_point=lambda: TimeSeriesEnv(data=X, reward_fn=mean_squared_error, expert_actions=Y, train_size=len_train),
    max_episode_steps=len_train,
)

rng = np.random.default_rng(0)
venv = make_vec_env(
    env_name="TimeSeriesEnv",
    rng=rng,
    n_envs=4,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],
)

from imitation.data.types import Transitions
flattened_transitions=Transitions(
    obs=X[:len_train], 
    acts=Y[:len_train], 
    infos=np.array([{} for _ in range(len_train)]),
    next_obs=X[:len_train],
    dones=np.array([True for _ in range(len_train)])
    )

flattened_transitions[0]

c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment TimeSeriesEnv already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\utils\passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed=412
  logger.warn(
c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not 

{'obs': array([ 1.50000000e+01,  1.63444714e-01,  5.90436631e+01,  3.31176955e-01,
         9.96200000e+00, -1.95747619e-02,  2.22815320e-01,  7.00000000e+00,
         1.50000000e+01,  7.00000000e+00,  7.42271121e+00]),
 'acts': 60,
 'infos': {},
 'next_obs': array([ 1.50000000e+01,  1.63444714e-01,  5.90436631e+01,  3.31176955e-01,
         9.96200000e+00, -1.95747619e-02,  2.22815320e-01,  7.00000000e+00,
         1.50000000e+01,  7.00000000e+00,  7.42271121e+00]),
 'dones': True}

In [22]:
from stable_baselines3 import PPO
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm

# 1) Expert rollouts already in `expert_rollouts`
# 2) Generator
gen_algo = PPO(
    policy="MlpPolicy",
    env=venv,
    batch_size=64,
    n_epochs=5,
    learning_rate=1e-4,
    gamma=0.99,
    seed=0,
)

# 3) Discriminator
disc_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)
# 4) GAIL trainer
gail = GAIL(
    demonstrations=flattened_transitions,
    demo_batch_size=1024,
    gen_algo=gen_algo,
    reward_net=disc_net,
    venv=venv,
)
gail.train(total_timesteps=200_000)
gail_policy = gail.gen_algo.policy


# register, vectorize, and wrap env to work with imitation apackage
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.util.util import make_vec_env

gym.register(
    id="TimeSeriesEnv",
    entry_point=lambda: TimeSeriesEnv(data=X, reward_fn=mean_squared_error, expert_actions=Y),
    max_episode_steps=len_train,
)

rng = np.random.default_rng(0)
venv = make_vec_env(
    env_name="TimeSeriesEnv",
    rng=rng,
    n_envs=4,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],
)

round:   0%|          | 0/1220 [00:00<?, ?it/s]c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
round:   0%|          | 0/1220 [00:00<?, ?it/s]


ValueError: could not broadcast input array from shape (5038,11) into shape (11,)